# RANKING: INSURANCE SECTOR

Profit:
- ROE
- ROA
- Combined Ratio

Health:
- Net Premium Written to Equity
- Net Leverage
- Gross Reserve to Equity
- Net Premium Written to Gross Premium Written

## 1. Import

### 1.1 Library

In [346]:
import pandas as pd
import numpy as np
import datetime as dt
import pyodbc
import pymssql
import sys

sys.path.append(r"E:\Tung\Python\DashBoard\vnd_data")
import get_vnd_data as vnd

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Customize the display of the table
pd.set_option('chained_assignment', None)

### 1.2 Import Data

In [347]:
# Assign pathlink
path_income_insurance = r"E:\Tung\Python\BSC_DataRankingStocks\cache\is_insurance.csv"
path_bs_insurance = r"E:\Tung\Python\BSC_DataRankingStocks\cache\bs_insurance.csv"

In [348]:
df_bs = pd.read_csv(path_bs_insurance)
df_bs.drop(['Unnamed: 0'], axis=1, inplace=True)

df_is = pd.read_csv(path_income_insurance)
df_is.drop(['Unnamed: 0'], axis=1, inplace=True)

# Preprocess data
df_is = df_is.loc[df_is['Quarter'] != 0]
df_bs = df_bs.loc[df_bs['Quarter'] != 0]
df_is.fillna(0, inplace=True)
df_bs.fillna(0, inplace=True)

# Sort data
df_bs.sort_values(by=['Symbol', 'Year', 'Quarter'], ascending=[True, True, True], inplace=True)
df_is.sort_values(by=['Symbol', 'Year', 'Quarter'], ascending=[True, True, True], inplace=True)

# Get stocks of this sectors
list_stocks = df_bs['Symbol'].unique()

In [349]:
df_is.info()
print("\n")
df_bs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 292 to 36
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Symbol               497 non-null    object 
 1   Year                 497 non-null    int64  
 2   Quarter              497 non-null    int64  
 3   GrossPremiumWritten  497 non-null    float64
 4   NetPremiumWritten    497 non-null    float64
 5   IncurredLosses       497 non-null    float64
 6   Expenses             497 non-null    float64
 7   NetIncome            497 non-null    float64
 8   NetIncome2           497 non-null    float64
dtypes: float64(6), int64(2), object(1)
memory usage: 38.8+ KB


<class 'pandas.core.frame.DataFrame'>
Int64Index: 532 entries, 317 to 35
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Symbol       532 non-null    object 
 1   Year         532 non-null    int64  
 2   Qu

#### 1.2.1 Check Null data

In [350]:
# df_is.isnull().any()
# df_bs.isnull().any()
# df_is[df_is.isna().any(axis=1)]
# df_bs[df_bs.isna().any(axis=1)]

#### 1.2.2 Import External Data
- Ceded Reserves: From VND's source due to SQL Server doesn't have this type of data

In [351]:
ceded_reserves = []
print("Get Ceded Reserve or 'Provision for claim from outward insurance' from the balance sheet ")
for i in list_stocks:
    print(f"Stock: {i}")
    df_i = vnd.get_balance_sheet(i)
    df_i['fiscalDate'] = pd.to_datetime(df_i['fiscalDate'])
    df_i['Year'] = df_i['fiscalDate'].dt.year
    df_i['Quarter'] = df_i['fiscalDate'].dt.quarter
    df_i = df_i.loc[df_i['itemCode'] == 411920]
    
    ceded_reserves.append(df_i)
    
print("Finish: Successfully get the data")
ceded_reserves = pd.concat(ceded_reserves)

# Process and Remove unnecessary columns

ceded_reserves.drop([
    'reportType', 'modelType', 'fiscalDate', 'createdDate', 'modifiedDate',
    'itemCode'
],
                          axis=1,
                          inplace=True)
ceded_reserves.rename(columns={
    "code": "Symbol",
    "numericValue": "CededReserves"
},
                            inplace=True)


Get Ceded Reserve or 'Provision for claim from outward insurance' from the balance sheet 
Stock: ABI
Stock: AIC
Stock: BIC
Stock: BLI
Stock: BMI
Stock: BVH
Stock: MIG
Stock: PGI
Stock: PRE
Stock: PTI
Stock: PVI
Stock: VNR
Finish: Successfully get the data


In [352]:
df_bs = df_bs.merge(ceded_reserves, how="inner", on=['Symbol', 'Year', 'Quarter'])

In [353]:
df_bs

,Symbol,Year,Quarter,Assets,Debt,Equity,Liabilities,Provisions,CededReserves
0,ABI,2015,1,1.167901e+12,0.0,5.290570e+11,6.388438e+11,4.565259e+11,4.749350e+10
1,ABI,2015,2,1.186682e+12,0.0,5.501613e+11,6.365207e+11,4.781257e+11,4.631121e+10
2,ABI,2015,3,1.249594e+12,0.0,5.703982e+11,6.791959e+11,5.019266e+11,3.923256e+10
3,ABI,2015,4,1.293413e+12,0.0,5.879564e+11,7.054566e+11,4.115277e+11,3.741064e+10
4,ABI,2016,1,1.370255e+12,0.0,6.094565e+11,7.607980e+11,5.830776e+11,4.068417e+10
...,...,...,...,...,...,...,...,...,...
361,VNR,2022,1,7.270789e+12,0.0,3.359424e+12,3.877725e+12,2.875009e+12,1.064367e+12
362,VNR,2022,2,7.590960e+12,0.0,3.258630e+12,4.298457e+12,2.933968e+12,1.060540e+12
363,VNR,2022,3,7.434190e+12,0.0,3.330368e+12,4.072038e+12,2.883861e+12,1.009162e+12
364,VNR,2022,4,7.126443e+12,0.0,3.453797e+12,3.640694e+12,2.740482e+12,9.017654e+11


- Net Revenues: From VND's source due to SQL Server has some errors in this type of data

In [354]:
net_revenue = []
print("Get Net revenue data from the income statement")
for i in list_stocks:
    print(f"Stock: {i}")
    df_i = vnd.get_income_statement(i)
    df_i['fiscalDate'] = pd.to_datetime(df_i['fiscalDate'])
    df_i['Year'] = df_i['fiscalDate'].dt.year
    df_i['Quarter'] = df_i['fiscalDate'].dt.quarter
    df_i = df_i.loc[df_i['itemCode'] == 21001]
    
    net_revenue.append(df_i)
    
print("Finish: Successfully get the data")
net_revenue = pd.concat(net_revenue)

# Process and Remove unnecessary columns

net_revenue.drop([
    'reportType', 'modelType', 'fiscalDate', 'createdDate', 'modifiedDate',
    'itemCode'
],
                          axis=1,
                          inplace=True)
net_revenue.rename(columns={
    "code": "Symbol",
    "numericValue": "Revenues"
},
                            inplace=True)


Get Net revenue data from the income statement
Stock: ABI
Stock: AIC
Stock: BIC
Stock: BLI
Stock: BMI
Stock: BVH
Stock: MIG
Stock: PGI
Stock: PRE
Stock: PTI
Stock: PVI
Stock: VNR
Finish: Successfully get the data


In [355]:
df_is = df_is.merge(net_revenue, how="inner", on=['Symbol', 'Year', 'Quarter'])

In [357]:
df_is.tail(5)

,Symbol,Year,Quarter,GrossPremiumWritten,NetPremiumWritten,IncurredLosses,Expenses,NetIncome,NetIncome2,Revenues
482,VNR,2022,1,0.0,3.304450e+11,7.805617e+10,2.679374e+11,8.353862e+10,8.362235e+10,3.909592e+11
483,VNR,2022,2,0.0,3.244055e+11,8.806094e+10,2.703435e+11,9.658117e+10,9.634965e+10,3.913818e+11
484,VNR,2022,3,0.0,3.295458e+11,9.828327e+10,2.450552e+11,7.159722e+10,7.368582e+10,3.874150e+11
485,VNR,2022,4,0.0,2.962349e+11,8.027482e+10,2.589780e+11,1.266493e+11,1.264806e+11,3.445524e+11
486,VNR,2023,1,0.0,4.301665e+11,1.172477e+11,3.228038e+11,2.091005e+11,2.097518e+11,4.981629e+11


#### 1.2.3 Preprocess data

- Combined Ratio (TTM)

In [358]:
def combined_ratio_ttm(panel_data, window=4) -> pd.DataFrame():
    """ This function is to calculate combined ratio TTM
    ================================================================
    panel_data: pd.DataFrame
    window: int
        Default value is 4
    """
    incurred_losses_ttm = panel_data.groupby('Symbol')['IncurredLosses'].rolling(window=window).sum().to_list()
    expenses_ttm = panel_data.groupby('Symbol')['Expenses'].rolling(window=window).sum().to_list()
    revenues_ttm = panel_data.groupby('Symbol')['Revenues'].rolling(window=window).sum().to_list()

    panel_data['incurred_losses_ttm'] = incurred_losses_ttm
    panel_data['expenses_ttm'] = expenses_ttm
    panel_data['revenues_ttm'] = revenues_ttm
    panel_data['combined_ratio_ttm'] = (panel_data['incurred_losses_ttm'] + panel_data['expenses_ttm'])/panel_data['revenues_ttm']
        
    
    return panel_data

In [359]:
df_is = combined_ratio_ttm(panel_data=df_is, window=4)

In [361]:
df_is.tail(3)
# df_is.info()

,Symbol,Year,Quarter,GrossPremiumWritten,NetPremiumWritten,IncurredLosses,Expenses,NetIncome,NetIncome2,Revenues,incurred_losses_ttm,expenses_ttm,revenues_ttm,combined_ratio_ttm
484,VNR,2022,3,0.0,3.295458e+11,9.828327e+10,2.450552e+11,7.159722e+10,7.368582e+10,3.874150e+11,3.841350e+11,1.075437e+12,1.581604e+12,0.922843
485,VNR,2022,4,0.0,2.962349e+11,8.027482e+10,2.589780e+11,1.266493e+11,1.264806e+11,3.445524e+11,3.446752e+11,1.042314e+12,1.514308e+12,0.915923
486,VNR,2023,1,0.0,4.301665e+11,1.172477e+11,3.228038e+11,2.091005e+11,2.097518e+11,4.981629e+11,3.838667e+11,1.097181e+12,1.621512e+12,0.913374


In [400]:
# Save data
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database - backup.accdb;'
)
is_insurance = pd.read_sql("SELECT * FROM income_statement_insurance", con=conn)
conn.commit()

In [405]:
df_is[['Symbol', 'Year', 'Quarter', 'incurred_losses_ttm', 'expenses_ttm', 'revenues_ttm']].astype(str)

,Symbol,Year,Quarter,incurred_losses_ttm,expenses_ttm,revenues_ttm
0,ABI,2012,2,nan,nan,nan
1,ABI,2012,3,nan,nan,nan
2,ABI,2012,4,nan,nan,nan
3,ABI,2013,1,129201316499.0,253830684223.0,397623137051.0
4,ABI,2013,2,134921638788.0,245840238595.0,411353175251.0
...,...,...,...,...,...,...
482,VNR,2022,1,416542938568.0,1137493327195.0,1607624361766.0
483,VNR,2022,2,398383042639.0,1105536823227.0,1585207852970.0
484,VNR,2022,3,384135039637.0,1075437374562.0,1581603961985.0
485,VNR,2022,4,344675196935.0,1042314142657.0,1514308357861.0


In [407]:
col_is_insurance = "],[".join(i for i in is_insurance.columns.to_list())
col_is_insurance

'Symbol],[Year],[Quarter],[incurred_losses_ttm],[expenses_ttm],[revenues_ttm'

In [424]:
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
cursor = conn.cursor()
for _, row in df_is[['Symbol', 'Year', 'Quarter', 'incurred_losses_ttm', 'expenses_ttm', 'revenues_ttm']].astype(str).iterrows():
    sql = "INSERT INTO income_statement_insurance (["+col_is_insurance+"]) VALUES "+ str(tuple(row))
    cursor.execute(sql)
    conn.commit()
    
print("Successfully saved data")


Successfully saved data


## 2. Process data

### 2.1 Profit Rank
- ROE
- ROA
- Combined Ratio

In [362]:
df_profit = pd.merge(
    df_bs[['Symbol', 'Year', 'Quarter', 'Equity', 'Assets']],
    df_is[['Symbol', 'Year', 'Quarter', 'NetIncome2', 'combined_ratio_ttm']],
    how='inner',
    on=['Symbol', 'Year', 'Quarter'])

In [363]:
# Check null or NaN values
df_profit[df_profit.isna().any(axis=1)]
df_profit.tail(3)

,Symbol,Year,Quarter,Equity,Assets,NetIncome2,combined_ratio_ttm
354,VNR,2022,3,3.330368e+12,7.434190e+12,7.368582e+10,0.922843
355,VNR,2022,4,3.453797e+12,7.126443e+12,1.264806e+11,0.915923
356,VNR,2023,1,3.661377e+12,7.738770e+12,2.097518e+11,0.913374


#### 2.1.1 Calculate ratios

In [364]:
df_profit['Equity_m'] = df_profit.groupby('Symbol')['Equity'].shift(4).to_list()
df_profit['Assets_m'] = df_profit.groupby('Symbol')['Assets'].shift(4).to_list()


df_profit['Equity_m'] = df_profit[['Equity', 'Equity_m']].mean(axis=1)
df_profit['Assets_m'] = df_profit[['Assets', 'Assets_m']].mean(axis=1)

df_profit['NetIncome2_ttm'] = df_profit.groupby('Symbol')['NetIncome2'].rolling(4).sum().to_list()

df_profit['ROE_ttm'] = df_profit['NetIncome2_ttm']/df_profit['Equity_m']
df_profit['ROA_ttm'] = df_profit['NetIncome2_ttm']/df_profit['Assets_m']

#### 2.1.2 Calcuate ratios of the Insurance sector

In [365]:
# Calculate ratios of the insurance sector
# Median approach
df_sector = df_profit.groupby(["Year", "Quarter"]).agg({
    "NetIncome2_ttm": "sum",
    "Equity_m": "sum",
    "Assets_m": "sum",
    'combined_ratio_ttm': "median"
}).reset_index()
df_sector['ROE_sector_ttm'] = df_sector['NetIncome2_ttm'] / df_sector['Equity_m']
df_sector['ROA_sector_ttm'] = df_sector['NetIncome2_ttm'] / df_sector['Assets_m']
df_sector.rename(columns={"combined_ratio_ttm": "combined_ratio_sector_ttm_median"}, inplace=True)

# Merge data from individual stocks and their sector
df_profit = pd.merge(df_profit,
                     df_sector[[
                         'Year', 'Quarter', 'ROE_sector_ttm', 
                         'ROA_sector_ttm', 'combined_ratio_sector_ttm_median']],
                     how='inner',
                     on=['Year', 'Quarter'])

In [366]:
df_is.columns

Index(['Symbol', 'Year', 'Quarter', 'GrossPremiumWritten', 'NetPremiumWritten',
       'IncurredLosses', 'Expenses', 'NetIncome', 'NetIncome2', 'Revenues',
       'incurred_losses_ttm', 'expenses_ttm', 'revenues_ttm',
       'combined_ratio_ttm'],
      dtype='object')

In [367]:
# Calculate ratios of the insurance sector
# Average approach
df_sector_avg_a = df_is.groupby(["Year", "Quarter"]).agg({
    "incurred_losses_ttm": "sum",
    "expenses_ttm": "sum",
    "revenues_ttm": "sum",
}).reset_index()

df_sector_avg_a['combined_ratio_sector_ttm_avg'] = (df_sector_avg_a['incurred_losses_ttm'] + df_sector_avg_a['expenses_ttm'])/df_sector_avg_a['revenues_ttm']


# Merge data from individual stocks and their sector
df_profit = pd.merge(df_profit,
                     df_sector_avg_a[[
                         'Year', 'Quarter', 'combined_ratio_sector_ttm_avg']],
                     how='inner',
                     on=['Year', 'Quarter'])

In [368]:
df_profit.loc[df_profit['Year'] == 2023]

,Symbol,Year,Quarter,Equity,Assets,NetIncome2,combined_ratio_ttm,Equity_m,Assets_m,NetIncome2_ttm,ROE_ttm,ROA_ttm,ROE_sector_ttm,ROA_sector_ttm,combined_ratio_sector_ttm_median,combined_ratio_sector_ttm_avg
307,ABI,2023,1,1.521999e+12,3.691074e+12,7.073732e+10,0.920597,1.440553e+12,3.533884e+12,2.365235e+11,0.164189,0.066930,0.077892,0.011895,0.989138,0.796984
308,AIC,2023,1,1.077122e+12,3.820574e+12,4.084042e+09,1.085106,1.072863e+12,3.747529e+12,6.805036e+09,0.006343,0.001816,0.077892,0.011895,0.989138,0.796984
309,BIC,2023,1,2.710626e+12,7.309509e+12,7.804477e+10,0.989138,2.675088e+12,6.776804e+12,3.186971e+11,0.119135,0.047028,0.077892,0.011895,0.989138,0.796984
310,BLI,2023,1,7.725179e+11,2.460191e+12,2.455057e+10,1.078538,8.164171e+11,2.461241e+12,-1.529464e+10,-0.018734,-0.006214,0.077892,0.011895,0.989138,0.796984
311,BMI,2023,1,2.536488e+12,7.402397e+12,7.349278e+10,0.968054,2.488836e+12,7.432981e+12,2.949475e+11,0.118508,0.039681,0.077892,0.011895,0.989138,0.796984
312,BVH,2023,1,2.108628e+13,2.204610e+14,5.282186e+11,0.648672,2.145454e+13,2.021196e+14,1.562514e+12,0.072829,0.007731,0.077892,0.011895,0.989138,0.796984
313,MIG,2023,1,1.966404e+12,8.705415e+12,6.866482e+10,0.992663,1.892244e+12,7.991447e+12,1.607458e+11,0.084950,0.020115,0.077892,0.011895,0.989138,0.796984
314,PGI,2023,1,1.835240e+12,6.875628e+12,5.488280e+10,0.947338,1.841491e+12,6.621079e+12,2.171827e+11,0.117939,0.032802,0.077892,0.011895,0.989138,0.796984
315,PRE,2023,1,9.262039e+11,6.318618e+12,3.651632e+10,1.212999,9.374136e+11,5.832837e+12,1.892971e+11,0.201936,0.032454,0.077892,0.011895,0.989138,0.796984
316,PTI,2023,1,1.848754e+12,8.797625e+12,4.110111e+10,1.088680,2.031787e+12,8.957038e+12,-3.494226e+11,-0.171978,-0.039011,0.077892,0.011895,0.989138,0.796984


#### 2.1.3 Scoring profit criteria

In [369]:
# Rank
df_profit['roe_score'] = np.where(df_profit['ROE_ttm'] > df_profit['ROE_sector_ttm'], 1, 0)
df_profit['roa_score'] = np.where(df_profit['ROA_ttm'] > df_profit['ROA_sector_ttm'], 1, 0)
df_profit['combineratio_score'] = np.where(df_profit['combined_ratio_ttm'] < df_profit['combined_ratio_sector_ttm_median'], 1, 0)
df_profit['profit_score'] = round((df_profit['roe_score']+df_profit['roa_score']+df_profit['combineratio_score'])*4/3,2)

rank_profit = []

for _, items in df_profit.iterrows():
    if items['profit_score'] < 1:
        rank_profit.append("D")
    elif items['profit_score'] < 2:
        rank_profit.append("C")
    elif items['profit_score'] < 3:
        rank_profit.append("B")
    else:
        rank_profit.append("A")

df_profit['rank_profit'] = rank_profit
df_profit.sort_values(by=['Symbol', 'Year', 'Quarter'], ascending=[True, True, True], inplace=True)
df_profit.tail(3)
        

,Symbol,Year,Quarter,Equity,Assets,NetIncome2,combined_ratio_ttm,Equity_m,Assets_m,NetIncome2_ttm,...,ROA_ttm,ROE_sector_ttm,ROA_sector_ttm,combined_ratio_sector_ttm_median,combined_ratio_sector_ttm_avg,roe_score,roa_score,combineratio_score,profit_score,rank_profit
296,VNR,2022,3,3.330368e+12,7.434190e+12,7.368582e+10,0.922843,3.269844e+12,7.373436e+12,3.248515e+11,...,0.044057,0.085942,0.014520,0.990267,0.850862,1,1,1,4.0,A
306,VNR,2022,4,3.453797e+12,7.126443e+12,1.264806e+11,0.915923,3.365918e+12,7.142578e+12,3.801384e+11,...,0.053221,0.075871,0.011946,0.988731,0.801540,1,1,1,4.0,A
317,VNR,2023,1,3.661377e+12,7.738770e+12,2.097518e+11,0.913374,3.510400e+12,7.504779e+12,5.062679e+11,...,0.067459,0.077892,0.011895,0.989138,0.796984,1,1,1,4.0,A


In [370]:
df_profit.loc[df_profit['Year'] == 2023]

,Symbol,Year,Quarter,Equity,Assets,NetIncome2,combined_ratio_ttm,Equity_m,Assets_m,NetIncome2_ttm,...,ROA_ttm,ROE_sector_ttm,ROA_sector_ttm,combined_ratio_sector_ttm_median,combined_ratio_sector_ttm_avg,roe_score,roa_score,combineratio_score,profit_score,rank_profit
307,ABI,2023,1,1.521999e+12,3.691074e+12,7.073732e+10,0.920597,1.440553e+12,3.533884e+12,2.365235e+11,...,0.066930,0.077892,0.011895,0.989138,0.796984,1,1,1,4.00,A
308,AIC,2023,1,1.077122e+12,3.820574e+12,4.084042e+09,1.085106,1.072863e+12,3.747529e+12,6.805036e+09,...,0.001816,0.077892,0.011895,0.989138,0.796984,0,0,0,0.00,D
309,BIC,2023,1,2.710626e+12,7.309509e+12,7.804477e+10,0.989138,2.675088e+12,6.776804e+12,3.186971e+11,...,0.047028,0.077892,0.011895,0.989138,0.796984,1,1,0,2.67,B
310,BLI,2023,1,7.725179e+11,2.460191e+12,2.455057e+10,1.078538,8.164171e+11,2.461241e+12,-1.529464e+10,...,-0.006214,0.077892,0.011895,0.989138,0.796984,0,0,0,0.00,D
311,BMI,2023,1,2.536488e+12,7.402397e+12,7.349278e+10,0.968054,2.488836e+12,7.432981e+12,2.949475e+11,...,0.039681,0.077892,0.011895,0.989138,0.796984,1,1,1,4.00,A
312,BVH,2023,1,2.108628e+13,2.204610e+14,5.282186e+11,0.648672,2.145454e+13,2.021196e+14,1.562514e+12,...,0.007731,0.077892,0.011895,0.989138,0.796984,0,0,1,1.33,C
313,MIG,2023,1,1.966404e+12,8.705415e+12,6.866482e+10,0.992663,1.892244e+12,7.991447e+12,1.607458e+11,...,0.020115,0.077892,0.011895,0.989138,0.796984,1,1,0,2.67,B
314,PGI,2023,1,1.835240e+12,6.875628e+12,5.488280e+10,0.947338,1.841491e+12,6.621079e+12,2.171827e+11,...,0.032802,0.077892,0.011895,0.989138,0.796984,1,1,1,4.00,A
315,PRE,2023,1,9.262039e+11,6.318618e+12,3.651632e+10,1.212999,9.374136e+11,5.832837e+12,1.892971e+11,...,0.032454,0.077892,0.011895,0.989138,0.796984,1,1,0,2.67,B
316,PTI,2023,1,1.848754e+12,8.797625e+12,4.110111e+10,1.088680,2.031787e+12,8.957038e+12,-3.494226e+11,...,-0.039011,0.077892,0.011895,0.989138,0.796984,0,0,0,0.00,D


### 2.2 Health Rank

#### 2.2.1 Calculate ratios

In [375]:
df_is.loc[(df_is['GrossPremiumWritten'] == 0) & (df_is['Year'] == 2019)]

,Symbol,Year,Quarter,GrossPremiumWritten,NetPremiumWritten,IncurredLosses,Expenses,NetIncome,NetIncome2,Revenues,incurred_losses_ttm,expenses_ttm,revenues_ttm,combined_ratio_ttm
470,VNR,2019,1,0.0,1.746626e+11,9.776917e+10,1.568028e+11,7.545732e+10,7.568332e+10,2.659463e+11,3.667895e+11,5.881292e+11,9.349605e+11,1.021346
471,VNR,2019,2,0.0,4.813893e+11,1.272594e+11,2.929253e+11,5.622506e+10,5.677511e+10,3.985788e+11,4.129132e+11,7.495740e+11,1.181667e+12,0.983768
472,VNR,2019,3,0.0,3.785740e+11,1.460964e+11,1.997340e+11,7.021839e+10,7.035755e+10,3.473131e+11,4.430736e+11,7.664005e+11,1.238330e+12,0.976698
473,VNR,2019,4,0.0,1.771382e+11,1.071933e+11,7.817427e+10,8.473219e+10,8.527868e+10,2.246997e+11,4.783183e+11,7.276364e+11,1.236538e+12,0.975267


In [376]:
df_health = pd.merge(df_is[['Symbol', 'Year', 'Quarter', 'GrossPremiumWritten','NetPremiumWritten']],
                     df_bs[['Symbol', 'Year', 'Quarter', 'Equity', 'Provisions', 'CededReserves']], 
                     how='inner',
                     on=['Symbol', 'Year', 'Quarter'])
df_health.sort_values(by=['Symbol', 'Year', 'Quarter'], ascending=[True, True, True], inplace=True)

# Calculate ratios
df_health['npw_to_equity'] = df_health['NetPremiumWritten']/df_health['Equity']
df_health['net_leverage'] = (df_health['NetPremiumWritten'] + df_health['Provisions'] - df_health['CededReserves'])/df_health['Equity']
df_health['gross_reserves_to_equity'] = df_health['Provisions']/df_health['Equity']
df_health['npw_gpw']= df_health['NetPremiumWritten']/df_health['GrossPremiumWritten']

#### 2.2.2 Function to score based on metrics

- Net Premium Written to Equity Score

In [377]:
def npw_to_equity_score(panel_data) -> pd.DataFrame:
    """ 
    This fuction is to calculate quantiles of Net Premium Written to Equity for each period of time, 
    after that, to score based on the quantitles.
    
    
    ================================================================
    panel_data: pd.DataFrame()
    """
    npw_to_equity = panel_data.groupby(['Year', 'Quarter'])['npw_to_equity'].quantile(0.25).reset_index(name='npw_to_equity_25')
    npw_to_equity['npw_to_equity_50'] = panel_data.groupby(['Year', 'Quarter'])['npw_to_equity'].quantile(0.50).to_list()
    npw_to_equity['npw_to_equity_75'] = panel_data.groupby(['Year', 'Quarter'])['npw_to_equity'].quantile(0.75).to_list()

    panel_data = pd.merge(
        panel_data,
        npw_to_equity,
        how='outer',
        on=['Year', 'Quarter']
    )

    npw_to_equity_score = []

    for _, items in panel_data.iterrows():
        if items['npw_to_equity'] < items['npw_to_equity_25']:
            npw_to_equity_score.append(4)
        elif items['npw_to_equity'] < items['npw_to_equity_50']:
            npw_to_equity_score.append(3)
        elif items['npw_to_equity'] < items['npw_to_equity_75']:
            npw_to_equity_score.append(2)
        else:
            npw_to_equity_score.append(1)
    panel_data['npw_to_equity_score'] = npw_to_equity_score
    
    del panel_data['npw_to_equity_25']
    del panel_data['npw_to_equity_50'] 
    del panel_data['npw_to_equity_75']
    
    
    return panel_data

- Net Leverage Score

In [378]:
def net_leverage_score(panel_data) -> pd.DataFrame:
    """ 
    This fuction is to calculate quantiles of Net Leverage for each period of time, 
    after that, to score based on the quantitles.
    ================================================================
    panel_data: pd.DataFrame()
    """
    net_leverage = panel_data.groupby(['Year', 'Quarter'])['net_leverage'].quantile(0.25).reset_index(name='net_leverage_25')
    net_leverage['net_leverage_50'] = panel_data.groupby(['Year', 'Quarter'])['net_leverage'].quantile(0.50).to_list()
    net_leverage['net_leverage_75'] = panel_data.groupby(['Year', 'Quarter'])['net_leverage'].quantile(0.75).to_list()

    panel_data = pd.merge(
        panel_data,
        net_leverage,
        how='outer',
        on=['Year', 'Quarter']
    )

    net_leverage_score = []

    for _, items in panel_data.iterrows():
        if items['net_leverage'] < items['net_leverage_25']:
            net_leverage_score.append(4)
        elif items['net_leverage'] < items['net_leverage_50']:
            net_leverage_score.append(3)
        elif items['net_leverage'] < items['net_leverage_75']:
            net_leverage_score.append(2)
        else:
            net_leverage_score.append(1)
    panel_data['net_leverage_score'] = net_leverage_score
    
    del panel_data['net_leverage_25']
    del panel_data['net_leverage_50'] 
    del panel_data['net_leverage_75']
    
    
    return panel_data

- Gross Reserves to Equity Score

In [379]:
def gross_reserves_to_equity_score(panel_data) -> pd.DataFrame:
    """ 
    This fuction is to calculate quantiles of Gross Reserve to Equity for each period of time, 
    after that, to score based on the quantitles.
    ================================================================
    panel_data: pd.DataFrame()
    """
    gross_reserves_to_equity = panel_data.groupby(['Year', 'Quarter'])['gross_reserves_to_equity'].quantile(0.25).reset_index(name='gross_reserves_to_equity_25')
    gross_reserves_to_equity['gross_reserves_to_equity_50'] = panel_data.groupby(['Year', 'Quarter'])['gross_reserves_to_equity'].quantile(0.50).to_list()
    gross_reserves_to_equity['gross_reserves_to_equity_75'] = panel_data.groupby(['Year', 'Quarter'])['gross_reserves_to_equity'].quantile(0.75).to_list()

    panel_data = pd.merge(
        panel_data,
        gross_reserves_to_equity,
        how='outer',
        on=['Year', 'Quarter']
    )

    gross_reserves_to_equity_score = []

    for _, items in panel_data.iterrows():
        if items['gross_reserves_to_equity'] < items['gross_reserves_to_equity_25']:
            gross_reserves_to_equity_score.append(4)
        elif items['gross_reserves_to_equity'] < items['gross_reserves_to_equity_50']:
            gross_reserves_to_equity_score.append(3)
        elif items['gross_reserves_to_equity'] < items['gross_reserves_to_equity_75']:
            gross_reserves_to_equity_score.append(2)
        else:
            gross_reserves_to_equity_score.append(1)
    panel_data['gross_reserves_to_equity_score'] = gross_reserves_to_equity_score
    
    del panel_data['gross_reserves_to_equity_25']
    del panel_data['gross_reserves_to_equity_50'] 
    del panel_data['gross_reserves_to_equity_75']
    
    
    return panel_data

- Net Premium Written to Gross Premium Written Score

In [380]:
def npw_gpw_score(panel_data) -> pd.DataFrame:
    """ 
    This fuction is to calculate quantiles of Net Premium Written to Gross Premium Written for each period of time, 
    after that, to score based on the quantitles.
    ================================================================
    panel_data: pd.DataFrame()
    """
    npw_gpw = panel_data.groupby(['Year', 'Quarter'])['npw_gpw'].quantile(0.25).reset_index(name='npw_gpw_25')
    npw_gpw['npw_gpw_50'] = panel_data.groupby(['Year', 'Quarter'])['npw_gpw'].quantile(0.50).to_list()
    npw_gpw['npw_gpw_75'] = panel_data.groupby(['Year', 'Quarter'])['npw_gpw'].quantile(0.75).to_list()

    panel_data = pd.merge(
        panel_data,
        npw_gpw,
        how='outer',
        on=['Year', 'Quarter']
    )

    npw_gpw_score = []

    for _, items in panel_data.iterrows():
        if items['npw_gpw'] < items['npw_gpw_25']:
            npw_gpw_score.append(4)
        elif items['npw_gpw'] < items['npw_gpw_50']:
            npw_gpw_score.append(3)
        elif items['npw_gpw'] < items['npw_gpw_75']:
            npw_gpw_score.append(2)
        else:
            npw_gpw_score.append(1)
    panel_data['npw_gpw_score'] = npw_gpw_score
    
    del panel_data['npw_gpw_25']
    del panel_data['npw_gpw_50'] 
    del panel_data['npw_gpw_75']
    
    
    return panel_data

#### 2.2.3 Scoring health criteria

In [381]:
df_health = npw_to_equity_score(panel_data=df_health)
df_health = net_leverage_score(panel_data=df_health)
df_health = gross_reserves_to_equity_score(panel_data=df_health)
df_health = npw_gpw_score(panel_data=df_health)
df_health['health_score'] = round(
    np.mean(df_health[[
        'npw_to_equity_score', 'net_leverage_score',
        'gross_reserves_to_equity_score', 'npw_gpw_score'
    ]],
            axis=1), 2)


In [382]:
rank_health = []

for _, items in df_health.iterrows():
    if items['health_score'] > 3:
        rank_health.append("Safe +")
    elif items['health_score'] > 2:
        rank_health.append("Safe")
    elif items['health_score'] > 1:
        rank_health.append("Warning")
    else:
        rank_health.append("Danger")
        
df_health['rank_health'] = rank_health
df_health.tail(3)

,Symbol,Year,Quarter,GrossPremiumWritten,NetPremiumWritten,Equity,Provisions,CededReserves,npw_to_equity,net_leverage,gross_reserves_to_equity,npw_gpw,npw_to_equity_score,net_leverage_score,gross_reserves_to_equity_score,npw_gpw_score,health_score,rank_health
354,VNR,2006,1,0.0,0.000000e+00,3.688930e+11,0.000000e+00,0.0,0.000000,0.000000,0.000000,NaN,1,1,1,1,1.0,Danger
355,VNR,2007,4,0.0,5.661399e+10,6.139692e+11,0.000000e+00,0.0,0.092210,0.092210,0.000000,inf,1,1,1,1,1.0,Danger
356,VNR,2008,3,0.0,8.105993e+10,1.890158e+12,2.807711e+11,0.0,0.042885,0.191429,0.148544,inf,1,1,1,1,1.0,Danger


In [383]:
# # Check columns
# df_profit.columns
df_health

,Symbol,Year,Quarter,GrossPremiumWritten,NetPremiumWritten,Equity,Provisions,CededReserves,npw_to_equity,net_leverage,gross_reserves_to_equity,npw_gpw,npw_to_equity_score,net_leverage_score,gross_reserves_to_equity_score,npw_gpw_score,health_score,rank_health
0,ABI,2015,1,1.485011e+11,1.526977e+11,5.290570e+11,4.565259e+11,4.749350e+10,0.288622,1.061757,0.862905,1.028260,2,3,4,1,2.50,Safe
1,BIC,2015,1,3.389173e+11,2.181472e+11,9.398289e+11,1.582981e+12,7.480503e+11,0.232114,1.120499,1.684329,0.643659,4,2,2,4,3.00,Safe
2,BMI,2015,1,7.440297e+11,6.085892e+11,2.214440e+12,2.077800e+12,6.227696e+11,0.274828,0.931892,0.938296,0.817964,2,4,4,3,3.25,Safe +
3,BVH,2015,1,3.523996e+12,3.336066e+12,1.267964e+13,2.856403e+13,7.642358e+11,0.263104,2.455580,2.252749,0.946671,3,1,1,2,1.75,Warning
4,PGI,2015,1,5.363529e+11,4.351534e+11,8.631015e+11,2.306856e+12,7.524167e+11,0.504174,2.305166,2.672751,0.811319,1,2,1,4,2.00,Warning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,PTI,2010,4,2.514455e+11,2.057941e+11,6.692808e+11,4.010235e+11,0.000000e+00,0.307485,0.906671,0.599186,0.818444,1,1,1,1,1.00,Danger
353,PTI,2011,2,2.536325e+11,2.042993e+11,6.368983e+11,5.044612e+11,0.000000e+00,0.320772,1.112831,0.792059,0.805493,1,1,1,1,1.00,Danger
354,VNR,2006,1,0.000000e+00,0.000000e+00,3.688930e+11,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,NaN,1,1,1,1,1.00,Danger
355,VNR,2007,4,0.000000e+00,5.661399e+10,6.139692e+11,0.000000e+00,0.000000e+00,0.092210,0.092210,0.000000,inf,1,1,1,1,1.00,Danger


## 3. Merge data

### 3.1 Merge profit & health
- `df_profit`: Profit
- `df_health`: Health

In [ ]:
# # # Export raw ratios
# pd.merge(
#     df_profit,
#     df_health,
#     how='inner',
#     on=['Symbol', 'Year', 'Quarter', 'Equity']
# ).to_excel(r'E:\Tung\Python\BSC_DataRankingStocks\data_raw_ratios\insurance_ratios.xlsx')

In [414]:
# Save stock_financial_ratio_insurance
df_sfri = pd.merge(
    df_profit[['Symbol', 'Year', 'Quarter', 'combined_ratio_ttm']],
    df_health[['Symbol', 'Year', 'Quarter', 'npw_to_equity', 'net_leverage', 'gross_reserves_to_equity', 'npw_gpw']],
    how='inner',
    on=['Symbol', 'Year', 'Quarter']
)
df_sfri

,Symbol,Year,Quarter,combined_ratio_ttm,npw_to_equity,net_leverage,gross_reserves_to_equity,npw_gpw
0,ABI,2015,1,0.831091,0.288622,1.061757,0.862905,1.028260
1,ABI,2015,2,0.827680,0.346608,1.131495,0.869065,0.948052
2,ABI,2015,3,0.844956,0.360402,1.171579,0.879958,0.998716
3,ABI,2015,4,0.917956,0.378204,1.014505,0.699929,0.971145
4,ABI,2016,1,0.922146,0.381543,1.271505,0.956717,0.966904
...,...,...,...,...,...,...,...,...
352,VNR,2022,1,0.966666,0.098364,0.637337,0.855804,inf
353,VNR,2022,2,0.948721,0.099553,0.674466,0.900369,inf
354,VNR,2022,3,0.922843,0.098952,0.661862,0.865929,inf
355,VNR,2022,4,0.915923,0.085771,0.618146,0.793469,inf


In [416]:
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database - backup.accdb;'
)
df_ratio_insurance = pd.read_sql("SELECT * FROM stock_financial_ratio_insurance", con=conn)
conn.commit()

In [420]:
col_df_sfri = "],[".join(i for i in df_ratio_insurance.columns.to_list())

In [425]:
col_df_sfri

'Symbol],[Year],[Quarter],[combined_ratio_ttm],[npw_to_equity],[net_leverage],[gross_reserve_to_equity],[npw_gpw'

In [426]:
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
cursor = conn.cursor()
for _, row in df_sfri.astype(str).iterrows():
    sql = "INSERT INTO stock_financial_ratio_insurance (["+col_df_sfri+"]) VALUES "+ str(tuple(row))
    cursor.execute(sql)
    conn.commit()
    
print("Successfully saved data")


Successfully saved data


In [427]:
df_final = pd.merge(
    df_profit[['Symbol', 'Year', 'Quarter', 'roe_score', 'roa_score', 'combineratio_score', 'profit_score', 'rank_profit']],
    df_health[['Symbol', 'Year', 'Quarter', 'npw_to_equity_score', 'net_leverage_score',
               'gross_reserves_to_equity_score', 'npw_gpw_score', 'health_score',
               'rank_health']],
    how='inner',
    on=['Symbol', 'Year', 'Quarter']
)

In [429]:
df_final

# df_final.info()

,Symbol,Year,Quarter,roe_score,roa_score,combineratio_score,profit_score,rank_profit,npw_to_equity_score,net_leverage_score,gross_reserves_to_equity_score,npw_gpw_score,health_score,rank_health
0,ABI,2015,1,0,0,1,1.33,C,2,3,4,1,2.50,Safe
1,ABI,2015,2,0,0,1,1.33,C,2,3,4,2,2.75,Safe
2,ABI,2015,3,0,0,1,1.33,C,1,2,4,1,2.00,Warning
3,ABI,2015,4,1,1,1,4.00,A,1,2,4,1,2.00,Warning
4,ABI,2016,1,1,1,1,4.00,A,1,2,3,1,1.75,Warning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,VNR,2022,1,0,1,0,1.33,C,4,4,4,1,3.25,Safe +
353,VNR,2022,2,1,1,0,2.67,B,4,4,4,1,3.25,Safe +
354,VNR,2022,3,1,1,1,4.00,A,4,4,4,1,3.25,Safe +
355,VNR,2022,4,1,1,1,4.00,A,4,4,4,1,3.25,Safe +


### 3.2 Import Raw data
To get growth score and valuation score

#### 3.2.1 Get the final result 
From table: `ptsp_stock_fundamental_score`

In [430]:
# Get raw final result from table: ptsp_stock_fundamental_score
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
df_raw = pd.read_sql('select * from ptsp_stock_fundamental_score', conn)

conn.close()

In [431]:
# Get the data of insurance sector
df_raw = df_raw.loc[df_raw['Symbol'].isin(list_stocks)]
df_raw[['Year', 'Quarter']] = df_raw[['Year', 'Quarter']].astype(int)

In [432]:
df_raw.tail(5)

,Symbol,Year,Quarter,Date,score_ROE_sector,score_ROA_sector,score_ROS_sector,score_ROE_group,score_ROA_group,score_ROS_group,...,rank_growth,score_PE_5Y,score_PB_5Y,score_PE_sector,score_PB_sector,score_valuation,rank_valuation,score_final,rank_final,Update
17520,PGI,2023,1,2023-05-15,1.0,1.0,1.0,0.0,1.0,0.0,...,D,0.0,1.0,0.0,2.0,1.5,C,2.125,B,20230516
17554,PRE,2023,1,2023-05-15,None,None,None,None,None,None,...,None,0.0,0.0,2.0,2.0,2.0,C,0.5,D,20230516
17565,PTI,2023,1,2023-05-15,0.0,0.0,0.0,0.0,0.0,0.0,...,D,None,2.0,None,1.0,1.5,C,1.625,C,20230516
17572,PVI,2023,1,2023-05-15,0.5,0.5,0.5,0.5,0.5,0.5,...,A,1.0,1.0,2.0,2.0,3.0,B,3.25,A,20230516
17811,VNR,2023,1,2023-05-15,None,None,None,None,None,None,...,None,2.0,2.0,0.0,0.0,2.0,C,0.5,D,20230516


#### 3.2.2 Merge data
It includes:
- New profit rank
- New health rank
- Current growth rank
- Current Valuation rank

In [433]:
df_final = pd.merge(df_final,
                    df_raw[[
                        'Symbol', 'Year', 'Quarter', 'score_EPS_above_average',
                        'score_EPS_growth', 'score_EPS_above_sector',
                        'score_EPS_above_group', 'score_growth', 'rank_growth',
                        'score_PE_5Y', 'score_PB_5Y', 'score_PE_sector',
                        'score_PB_sector', 'score_valuation', 'rank_valuation',
                        'score_final', 'rank_final', 'Update'
                    ]],
                    how='inner',
                    on=['Symbol', 'Year', 'Quarter'])


In [434]:
# Change type of data in order to calculate
list_col = [
    'roe_score',
    'roa_score',
    'combineratio_score',
    'profit_score',
    'npw_to_equity_score',
    'net_leverage_score',
    'gross_reserves_to_equity_score',
    'npw_gpw_score',
    'health_score',
    'score_EPS_above_average',
    'score_EPS_growth',
    'score_EPS_above_sector',
    'score_EPS_above_group',
    'score_growth',
    'score_PE_5Y',
    'score_PB_5Y',
    'score_PE_sector',
    'score_PB_sector',
    'score_valuation',
]

for i in list_col:
    df_final[i] = df_final[i].astype(float)

### 3.3 Calculate Final Score

In [435]:
df_final['score_final'] = round(
    np.mean(df_final[[
        'profit_score', 'health_score', 'score_growth', 'score_valuation'
    ]],
            axis=1), 2)

for _, items in df_final.iterrows():
    if items['score_final'] < 1:
        items['rank_final'] = "D"
    elif items['score_final'] < 2:
        items['rank_final'] = "C"
    elif items['score_final'] < 3:
        items['rank_final'] = "B" 
    else:
        items['rank_final'] = "A"

In [436]:
df_final

,Symbol,Year,Quarter,roe_score,roa_score,combineratio_score,profit_score,rank_profit,npw_to_equity_score,net_leverage_score,...,rank_growth,score_PE_5Y,score_PB_5Y,score_PE_sector,score_PB_sector,score_valuation,rank_valuation,score_final,rank_final,Update
0,ABI,2019,1,1.0,1.0,1.0,4.00,A,1.0,2.0,...,A,2.0,2.0,0.0,0.0,2.0,C,2.62,B,20220422
1,ABI,2019,2,1.0,1.0,1.0,4.00,A,1.0,2.0,...,A,2.0,2.0,0.0,0.0,2.0,C,2.62,B,20220422
2,ABI,2019,3,1.0,1.0,1.0,4.00,A,1.0,2.0,...,A,2.0,2.0,0.0,0.0,2.0,C,2.88,B,20220422
3,ABI,2019,4,1.0,1.0,1.0,4.00,A,3.0,2.0,...,C,2.0,2.0,0.0,0.0,2.0,C,2.31,B,20220422
4,ABI,2020,1,1.0,1.0,1.0,4.00,A,1.0,2.0,...,A,2.0,2.0,0.0,0.0,2.0,C,2.69,B,20220422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,VNR,2022,1,0.0,1.0,0.0,1.33,C,4.0,4.0,...,D,1.0,1.0,0.0,0.0,1.0,D,1.40,C,20220628
164,VNR,2022,2,1.0,1.0,0.0,2.67,B,4.0,4.0,...,B,1.0,1.0,0.0,0.0,1.0,D,2.23,C,20221215
165,VNR,2022,3,1.0,1.0,1.0,4.00,A,4.0,4.0,...,C,2.0,2.0,0.0,0.0,2.0,C,2.56,C,20221227
166,VNR,2022,4,1.0,1.0,1.0,4.00,A,4.0,4.0,...,A,2.0,2.0,0.0,0.0,2.0,C,3.06,B,20230328


## 4. Save to DB Access

### 4.1 Get data fields in new table
- `ptsp_stock_fundamental_score_financial`

In [437]:
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
df_db = pd.read_sql("SELECT * FROM ptsp_stock_fundamental_score_financial", con=conn)
conn.commit()

In [438]:
# "],[".join(i for i in df_db.columns.to_list())
col_df_db = '[Symbol],[Year],[Quarter],[score_roe_sector],[score_roa_sector],[score_combined_ratio_sector],[score_profit],[rank_profit],[score_npw2equity],[score_net_leverage],[score_grossreserve2equity],[score_npw2gpw],[score_health],[rank_health],[score_EPS_above_average],[score_EPS_growth],[score_EPS_above_sector],[score_EPS_above_group],[score_growth],[rank_growth],[score_PE_5Y],[score_PB_5Y],[score_PE_sector],[score_PB_sector],[score_valuation],[rank_valuation],[score_final],[rank_final],[update]'

### 4.2 Save data to new table

In [439]:
conn = pyodbc.connect(
    r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=V:\iBroker\stock_database.accdb;'
)
cursor = conn.cursor()
for _, row in df_final.astype(str).iterrows():
    sql = "INSERT INTO ptsp_stock_fundamental_score_financial ("+col_df_db+") VALUES "+ str(tuple(row))
    cursor.execute(sql)
    conn.commit()
    
print("Successfully saved data")


Successfully saved data


In [ ]:
""" NOTES: 
From now, this script need to be upgraded in step Saving data to new table. 
Because, it insert new data will get trouble based on current situation"""